In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import Levenshtein
import re
from  translate import Translator

translator = Translator(to_lang="en")


In [36]:
unique = pd.read_csv("../dataset/result_cleaning_data.csv")
data = pd.read_csv("../dataset/cleaned_data.csv")

print(unique.head())
print(data.head())

C:\Users\USER\AppData\Local\Temp\ipykernel_17792\1811373139.py:2: DtypeWarning: Columns (194,195,196,250,258,261,262,263,270,271,272,273,274,275,317,330,332,333,337,338,339,340,344,345,346,347,351,352,353,354,358,359,360,361,365,366,367,368,372,373,374,375,379,380,381,382,384,385,386,387,389,390,408,460,469,478,485,487,488,494,495,496,497,498,503,504,505,506,507,512,513,514,515,516,521,522,523,524,525,528,529,530,531,532,613,622,624,625,626,646,647,648,649,650,651,659,660,661,662,663,668,669,670,671,672,677,678,679,680,681,691,692,696,697,702,703,704,709,710,711,718,719,720,727,728,729,736,737,738,746,747,748,749,750,755,756,757,758,759,764,765,766,767,768,773,774,775,776,777,782,783,784,785,786,791,792,793,794,795,800,801,802,803,804,809,810,811,812,813,818,819,820,821,822,827,828,829,830,831,836,837,838,839,840,845,846,847,848,849,850,851,860,861,862,866,867,868,872,873,874,875,876,877,878,879,880,881,882,883,884,885,899,900,901,906,907,908,913,914,915,920,921,922,927,928,929,934,935

   Unnamed: 0                      0                  1           2  \
0           0      software engineer             intern     manager   
1           1       computer science            general  management   
2           2  boy scouts of america           sciences  technology   
3           3  boy scouts of america           sciences  technology   
4           4                   inc.  software engineer       owner   

                         3                        4                       5  \
0       research assistant                 director         project manager   
1  business administration                  finance  mechanical engineering   
2        purdue university  the university of texas           college board   
3        purdue university  the university of texas           college board   
4                      llc                  founder                director   

                        6                         7                         8  \
0                

**experiences/{j}/title**

In [43]:
data = data[[f"experiences/{i}/title"  if i > 0 else "full_name" for i in range(83)]]

data

,full_name,experiences/1/title,experiences/2/title,experiences/3/title,experiences/4/title,experiences/5/title,experiences/6/title,experiences/7/title,experiences/8/title,experiences/9/title,...,experiences/73/title,experiences/74/title,experiences/75/title,experiences/76/title,experiences/77/title,experiences/78/title,experiences/79/title,experiences/80/title,experiences/81/title,experiences/82/title
0,Jeff B.,"AVP, Underwriting Operations","Manager, Underwriting Administration",Business Analyst,Supervisor,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Andrew Bailey,Associate,Scientific and Regulatory Affairs Manager,Scientific and Regulatory Affairs Coordinator,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alex Cabrera,Research Intern,Software Engineering Intern,Software Engineering Intern,Engineering Practicum Intern,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alcino de Oliveira Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ali Farhat,Junior Researcher,R&D Consultant,CAD designer,R&D Engineer,Teaching Assistant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8934,Aaron Dolores,Founder | Creative Director,Co-Founder,"Co-Head of Facebook's ""Oakland"" Developer's Ci...",Head Cultural Strategist,Account Executive,Account Executive,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8935,"Aaron Dominy, MBA",Senior Management Consultant- Oil and Gas,OEM Business Planning Strategy Intern,Management Consultant,"Buyer, Indirect Procurement- Contracts and Ser...",Contract Analyst,Transportation Sales Consultant,Customer Service Representative,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8936,Aaron Williams,Community Organizer,Teacher Mentor/Section Leader- Volunteer,LF Edge- Developer Advocate/ Technical Evangel...,Community Lead/ Developer Relations,d-shop Global Director/ Developer Advocate,Speaking/ Teaching Events,Lead- d-shop Americas,Product Manager- SAP Hana Cloud Platform- Inte...,Project Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8937,Aaron Dona,Level 3 Quality Assurance Tester,Hardware Tester,Game Tester (On contract through Volt Workforc...,Academic Coordinator,Assistant Director,Curriculum Developer,Instructor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
colomns = unique.iloc[0].dropna()
colomns

Unnamed: 0                                           0
0                                    software engineer
1                                               intern
2                                              manager
3                                   research assistant
                                ...                   
6059                            lead network architect
6060          satellite communication systems operator
6061                                 electrical intern
6062                     talent acquisition consultant
6063                 seasonal parks maintenance worker
Name: 0, Length: 6065, dtype: object

Pencarian kedekatan dengan menggunakan cosine similarity\
Cosine Similarity mengukur kesamaan antara dua vektor dengan mengukur sudut kosinus di antara keduanya. Dengan menggunakan TF-IDF (Term Frequency-Inverse Document Frequency), Anda bisa mengonversi kalimat menjadi vektor yang merepresentasikan pentingnya kata-kata dalam kalimat tersebut.

In [39]:

def most_similarity(input_word, data):
    most_similar_word = None
    highest_similarity = -1

    for word in data:
        distance = Levenshtein.distance(input_word, word)
        similarity = 1 - (distance / max(len(input_word), len(word)))

        if similarity > highest_similarity:
            highest_similarity = similarity
            most_similar_word = word

    return most_similar_word

input_word = "kardi"

sample = ["Kardus", "Karpet", "Karikatur"]

output = most_similarity(input_word, sample)

print(output)  


Kardus


In [58]:
def extract_each_title(title_list):
    exp_titles = []
    for exp_title in title_list:
        parts = str(exp_title).lower().replace("and", ",").replace("|", ",").replace("/", ",").replace("&", ",")\
            .replace(" – ", ",").replace(" - ", ",").split(",")
        
        parts = [part.strip() for part in parts]
        
        parts = [part.split(" at ")[0].strip() for part in parts]
        parts = [part.split(" on ")[0].strip() for part in parts]
        parts = [part for part in parts if len(part) > 1]
        parts = [re.sub(r"[()]", "", part) for part in parts]
        parts = [translator.translate(part) for part in parts]
        
        exp_titles.extend(parts)
    # print(exp_titles)
    return exp_titles

title_each_people = [extract_each_title(data.drop(columns="full_name").iloc[i].dropna()) for i in range(len(data))]
title_each_people


[['avp',
  'underwriting operations',
  'manager',
  'underwriting administration',
  'business analyst',
  'supervisor'],
 ['associate',
  'scientific',
  'regulatory affairs manager',
  'scientific',
  'regulatory affairs coordinator'],
 ['research intern',
  'software engineering intern',
  'software engineering intern',
  'engineering practicum intern'],
 [],
 ['junior researcher',
  'd consultant',
  'cad designer',
  'd engineer',
  'teaching assistant'],
 ['adjoint administratif', 'étudiant tuteur', 'stagiaire business assistant'],
 ['actor',
  'dramaturge',
  'verse coach',
  'editor',
  'copy writer',
  'staff writer',
  'developmental writing professor',
  'grant writer',
  'researcher'],
 ['volunteer', 'coffee clerk', 'coffee clerk'],
 ['teaching assistant', 'research assistant', 'photovoltaics lab in-charge'],
 [],
 ['mobile xamarin developer',
  'mobile xamarin developer',
  'mobile lead developer',
  'lead developer',
  'xamarin developer'],
 ['industrial engineer', 'mech

In [69]:
title_each_people = [title for title in title_each_people if len(title) != 0]
title_each_people

[['avp',
  'underwriting operations',
  'manager',
  'underwriting administration',
  'business analyst',
  'supervisor'],
 ['associate',
  'scientific',
  'regulatory affairs manager',
  'scientific',
  'regulatory affairs coordinator'],
 ['research intern',
  'software engineering intern',
  'software engineering intern',
  'engineering practicum intern'],
 ['junior researcher',
  'd consultant',
  'cad designer',
  'd engineer',
  'teaching assistant'],
 ['adjoint administratif', 'étudiant tuteur', 'stagiaire business assistant'],
 ['actor',
  'dramaturge',
  'verse coach',
  'editor',
  'copy writer',
  'staff writer',
  'developmental writing professor',
  'grant writer',
  'researcher'],
 ['volunteer', 'coffee clerk', 'coffee clerk'],
 ['teaching assistant', 'research assistant', 'photovoltaics lab in-charge'],
 ['mobile xamarin developer',
  'mobile xamarin developer',
  'mobile lead developer',
  'lead developer',
  'xamarin developer'],
 ['industrial engineer', 'mechanical eng

In [73]:
np.zeros(len(colomns))

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
def sentences_count(titles):
    frequency = np.zeros(len(colomns))

